<a href="https://colab.research.google.com/github/ElizabethHaberstroh/Oracular-Spectacular/blob/main/get_inventory_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Inventory.rtf to Inventory (7).rtf


In [ ]:
!pip -q install striprtf

In [ ]:
import json, pandas as pd, re
from striprtf.striprtf import rtf_to_text

In [ ]:
rtf_file = list(uploaded.keys())[0]
raw = open(rtf_file, 'r').read()
text = rtf_to_text(raw)

In [ ]:
l = text.find('[')
r = text.rfind(']')
json_text = text[l:r+1]

In [ ]:
json_text = re.sub(r'[\t]+', '', json_text)
json_text = re.sub(r'[\n]+', '', json_text)

In [ ]:
data = json.loads(json_text)
df = pd.json_normalize(data)

In [ ]:
out_csv = "Inventory_clean.csv"
df.to_csv(out_csv, index=False)

files.download(out_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_csv("Inventory_clean.csv")


In [ ]:
# Remove italic formatting
def remove_italics(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r"\*(.*?)\*", r"\1", text)              # remove *...*
    text = re.sub(r"</?i>", "", text, flags=re.IGNORECASE) # remove <i> tags
    text = re.sub(r"</?em>", "", text, flags=re.IGNORECASE) # remove <em> tags
    return text

# Apply cleaning to all string values
df = df.map(remove_italics)

# Clean HTML tags
def clean_html(text):
    if not isinstance(text, str):
        return text
    # Replace <br> (and variants like <br/>, <br />) with a space
    text = re.sub(r"<br\s*/?>", " ", text, flags=re.IGNORECASE)
    # Remove all other HTML tags
    text = re.sub(r"<[^>]+>", "", text)
    # Optional: strip leading/trailing whitespace
    text = text.strip()
    return text

# Apply cleaning to all string values
df = df.map(clean_html)

# Keep only needed columns
needed_columns = ["bottle_size", "country", "name", "price", "producer", "region", "varietal", "farming_type", "vintage", "wine_type"]
df = df[needed_columns]

# Rename columns
rename_map = {
    "country": "country_name",
    "name": "wine_name",
    "producer": "producer_name",
    "region": "region_name",
    "varietal": "grape_varietal"
}
df = df.rename(columns=rename_map)

#Select types of wines
allowed_types = ["WHITE", "RED", "SPARKLING", "ROSE"]
df = df[df["wine_type"].isin(allowed_types)]

#Split multi valued cells
df = df.assign(grape_varietal=df["grape_varietal"].str.split(", "))
df = df.explode("grape_varietal")

df = df.dropna(how="any")

print(df)

    bottle_size country_name  \
0         750mL       France   
1         750mL       France   
3         750mL       France   
4         750mL        Italy   
5         750mL        Italy   
..          ...          ...   
872       750mL       France   
873       750mL        Italy   
874       750mL       France   
875       750mL       France   
876       750mL       France   

                                             wine_name  price  \
0      2022 Saint-Véran “Les Pommards Vieilles Vignes”   55.0   
1                          2022 Riesling “Clos Mathis”   62.0   
3                                    2020 Bandol Rouge   52.0   
4                2023 Erbaluce di Caluso “Tredicimesi”   47.0   
5    2020 Friuli Colli Orientali Refosco Peduncolo ...   58.0   
..                                                 ...    ...   
872             2018 Vin de France “Grenache Oxydatif”   69.0   
873                 2019 Alto Adige Sauvignon “Voglar”   39.0   
874         2019 Gewurztramine

In [ ]:
# Save to CSV with UTF-8 preserved
output_filename = "get_inventory_3.csv"
df.to_csv(output_filename, index=False, encoding="utf-8-sig")

# Download the file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>